In [1]:
import os
import time
import random
import itertools
import subprocess
import distutils.util

import numpy as np
import imageio
import gymnasium as gym
import robosuite as suite
from robosuite.wrappers import GymWrapper


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


In [2]:
# For Ubuntu
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# For mac
#os.environ["MUJOCO_GL"] = "cgl"   # macOS

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')



# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

# For macOS

In [ ]:

# Create environment with Franka rob1t
env = suite.make(
    env_name="Lift",
    robots="Panda",
    has_renderer=False,
    has_offscreen_renderer=True,
    use_camera_obs=False
)

# Reset the environment
obs = env.reset()

# Get cube position from observation
cube_pos = obs['cube_pos']
print("Cube position:", cube_pos)

# Define target positions for lifting the cube:
# Stage 1: Move above the cube
above_cube_pos = cube_pos.copy()
above_cube_pos[2] += 0.1  # Position 10cm above the cube

# Stage 2: Move down to grasp the cube
grasp_pos = cube_pos.copy()
grasp_pos[2] -= 0.01  # Move slightly down to grasp

# Stage 3: Lift the cube
lift_pos = cube_pos.copy()
lift_pos[2] += 0.5  # Lift 20cm above the table
#lift_pos[0] += 0.5  # Lift 20cm above the table

print(f"Above cube target: {above_cube_pos}")
print(f"Grasp position: {grasp_pos}")
print(f"Lift position: {lift_pos}")

# Control gains
kp = 5.0  # Position gain
gripper_close_steps = 50  # Steps over which to close gripper gradually
kd = 1.0  # Velocity gain

# Run the simulation
done = False
step_count = 0
max_steps = 500
current_stage = 1
grasped = False

frames = []
while step_count < max_steps:
    current_eef = obs['robot0_eef_pos']
    current_cube_pos = obs['cube_pos']

    # Determine target
    if current_stage == 1:
        target = above_cube_pos
        if np.linalg.norm(current_eef - target) < 0.02:
            current_stage = 2
            print("Above cube reached, moving to grasp")
    elif current_stage == 2:
        target = grasp_pos
        if np.linalg.norm(current_eef - target) < 0.01:
            current_stage = 3
            print("ready to grasp")
    elif current_stage == 3:
        target = grasp_pos
        if action[-1] > 0.5:
            grasped = True
        if np.linalg.norm(current_eef - target) < 0.01 and grasped:
            current_stage = 4 
            print("Cube grasped, lifting")
    else:
        target = lift_pos
        if np.linalg.norm(current_cube_pos - target) < 0.01:
            print("Cube lifted successfully!, current_cube_pos:", current_cube_pos, ", target:", target, ", current_stage:", current_stage)
            break

    # Compute simple proportional action in end-effector space
    action = np.zeros(env.action_dim)
    if current_stage <= 2:
        action[:3] = kp * (target - current_eef)  # x,y,z
        action[3:6] = 0.0  # no rotation change
    else:
        action[:3] = kp * (target - current_cube_pos)  # x,y,z
        action[3:6] = 0.0  # no rotation change
    # Gripper control
    if current_stage <= 2:
        action[-1] = -1.0  # open
    else:
        # Gradually close over first gripper_close_steps
        action[-1] = min(1.0, (step_count % gripper_close_steps) / gripper_close_steps)

    # Step environment
    obs, reward, done, info = env.step(action)
    # Render offscreen frame
    frame = env.sim.render(camera_name="frontview", width=640, height=480)
    frame = np.flip(frame, axis=0)
    frames.append(frame)

    time.sleep(0.01)
    step_count += 1

print("cube_pos:", current_cube_pos, "current_eef:", current_eef, ", target:", target, ", current_stage:", current_stage)
env.close()
print("Simulation finished")
imageio.mimwrite("lift_cube.mp4", frames, fps=30)
print("Video saved!")

[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


Cube position: [-0.024  0.019  0.83 ]
Above cube target: [-0.024  0.019  0.93 ]
Grasp position: [-0.024  0.019  0.82 ]
Lift position: [-0.024  0.019  1.33 ]
Above cube reached, moving to grasp
ready to grasp
Cube grasped, lifting
Cube lifted successfully!, current_cube_pos: [-0.025  0.02   1.321] , target: [-0.024  0.019  1.33 ] , current_stage: 4
cube_pos: [-0.025  0.02   1.321] current_eef: [-0.029  0.02   1.33 ] , target: [-0.024  0.019  1.33 ] , current_stage: 4
Simulation finished
Video saved!


#### Robosuite envs aren’t directly Gym-compatible, use its GymWrapper and randomize parameters.

In [3]:
class RandomizedLiftEnv(gym.Wrapper):
    """
    Robosuite Lift task with domain randomization:
    - Random cube mass
    - Random cube friction
    """
    def __init__(self):
        env = suite.make(
            env_name="Lift",
            robots="Panda",
            has_renderer=False,
            has_offscreen_renderer=True,
            use_camera_obs=False,
            horizon=200,
            control_freq=20,
            reward_shaping=True,
        )
        super().__init__(GymWrapper(env))

    def reset(self, **kwargs):
        # Sample random parameters
        mass = np.random.uniform(0.05, 0.3)      # 50g – 300g
        friction = np.random.uniform(0.2, 1.0)   # slippery – sticky

        # Access robosuite env from inside wrapper
        env = self.env.env  

        # ✅ Body for mass
        cube_body_id = env.sim.model.body_name2id("cube_main")
        env.sim.model.body_mass[cube_body_id] = mass

        # ✅ Geom for friction (use cube_g0, not cube_main)
        cube_geom_id = env.sim.model.geom_name2id("cube_g0")
        env.sim.model.geom_friction[cube_geom_id] = [friction, 0.005, 0.0001]

        return super().reset(**kwargs)


In [17]:
from stable_baselines3 import PPO

env = RandomizedLiftEnv()

model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./ppo_franka_lift/")
model.learn(total_timesteps=200_000)
model.save("ppo_franka_lift")


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./ppo_franka_lift/PPO_1


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 2.54     |
| time/              |          |
|    fps             | 299      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 44.1        |
| time/                   |             |
|    fps                  | 282         |
|    iterations           | 96          |
|    time_elapsed         | 694         |
|    total_timesteps      | 196608      |
| train/                  |             |
|    approx_kl            | 0.018396363 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.2         |
|    entropy_loss         | -9.16       |
|    explained_variance   | 0.822       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.696       |
|    n_updates            | 950         |
|    policy_gradient_loss | -0.0237     |
|    std                  | 0.897       |
|    value_loss           | 2.41        |
-----------------------------------------


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 43.8        |
| time/                   |             |
|    fps                  | 282         |
|    iterations           | 97          |
|    time_elapsed         | 702         |
|    total_timesteps      | 198656      |
| train/                  |             |
|    approx_kl            | 0.017100502 |
|    clip_fraction        | 0.219       |
|    clip_range           | 0.2         |
|    entropy_loss         | -9.14       |
|    explained_variance   | 0.713       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.1         |
|    n_updates            | 960         |
|    policy_gradient_loss | -0.0173     |
|    std                  | 0.893       |
|    value_loss           | 3.68        |
-----------------------------------------


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | 43.8        |
| time/                   |             |
|    fps                  | 282         |
|    iterations           | 98          |
|    time_elapsed         | 709         |
|    total_timesteps      | 200704      |
| train/                  |             |
|    approx_kl            | 0.025850555 |
|    clip_fraction        | 0.226       |
|    clip_range           | 0.2         |
|    entropy_loss         | -9.11       |
|    explained_variance   | 0.83        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.575       |
|    n_updates            | 970         |
|    policy_gradient_loss | -0.0214     |
|    std                  | 0.891       |
|    value_loss           | 2.43        |
-----------------------------------------


In [21]:
import imageio

env = RandomizedLiftEnv()
# model = PPO.load("ppo_franka_lift")
model = PPO.load("./ppo_franka_lift.zip")


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [ ]:
# # Create env with domain randomization
# env = RandomizedLiftEnv()

# frames = []
# obs, _ = env.reset()
# done = False

# while not done:
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     done = terminated or truncated

#     frame = env.env.sim.render(camera_name="frontview", width=640, height=480)
#     frame = np.flip(frame, axis=0)   # flip because MuJoCo’s render is upside down
#     frames.append(frame)

# env.close()
# imageio.mimsave("ppo_eval.mp4", frames, fps=30)
# print("Evaluation finished, saved to ppo_eval.mp4")

[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


NameError: name 'model' is not defined

#### Issue: PPO agent is trained to mimic behavior from random exploration and the default reward shaping, but it was not explicitly trained with staged grasp-and-lift rewards

In [4]:
class RandomizedLiftEnv(gym.Wrapper):
    """
    Robosuite Lift task with domain randomization:
    - Random cube mass and friction
    - Reward shaping to encourage lifting
    """
    def __init__(self):
        env = suite.make(
            env_name="Lift",
            robots="Panda",
            has_renderer=False,
            has_offscreen_renderer=True,
            use_camera_obs=False,
            reward_shaping=True,  # gives intermediate reward for lifting
        )
        super().__init__(GymWrapper(env))

    def reset(self, **kwargs):
        # Domain randomization
        mass = np.random.uniform(0.05, 0.3)      # 50g – 300g
        friction = np.random.uniform(0.2, 1.0)   # slippery – sticky
        env = self.env.env

        # Mass
        cube_body_id = env.sim.model.body_name2id("cube_main")
        env.sim.model.body_mass[cube_body_id] = mass

        # Friction
        cube_geom_id = env.sim.model.geom_name2id("cube_g0")
        env.sim.model.geom_friction[cube_geom_id] = [friction, 0.005, 0.0001]

        return super().reset(**kwargs)

In [5]:
# Reward wrapper to boost lifting incentive
class LiftRewardWrapper(gym.RewardWrapper):
    def reward(self, reward):
        # Extra reward proportional to cube height
        cube_height = self.env.env.sim.data.get_body_xpos("cube_main")[2]
        reward += 2.0 * max(0, cube_height - 0.05)  # bonus for lifting above table
        return reward


In [6]:
from stable_baselines3.common.vec_env import DummyVecEnv

env = DummyVecEnv([lambda: LiftRewardWrapper(RandomizedLiftEnv())])


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/gymnasium/spaces/box.py:236:

In [7]:
from stable_baselines3 import PPO

model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    tensorboard_log="./ppo_franka_lift/",
    batch_size=256,
    learning_rate=3e-4,
    n_steps=2048,
)

# Train for enough timesteps to learn grasp + lift
model.learn(total_timesteps=500_000)
model.save("ppo_franka_lift_lifted")


Using cuda device


/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


Logging to ./ppo_franka_lift/PPO_1


[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


-----------------------------
| time/              |      |
|    fps             | 94   |
|    iterations      | 1    |
|    time_elapsed    | 21   |
|    total_timesteps | 2048 |
-----------------------------


...

-----------------------------------------
| time/                   |             |
|    fps                  | 98          |
|    iterations           | 243         |
|    time_elapsed         | 5059        |
|    total_timesteps      | 497664      |
| train/                  |             |
|    approx_kl            | 0.012333941 |
|    clip_fraction        | 0.13        |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.5       |
|    explained_variance   | -0.00311    |
|    learning_rate        | 0.0003      |
|    loss                 | 70.3        |
|    n_updates            | 2420        |
|    policy_gradient_loss | -0.0132     |
|    std                  | 1.26        |
|    value_loss           | 168         |
-----------------------------------------


[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


-----------------------------------------
| time/                   |             |
|    fps                  | 98          |
|    iterations           | 244         |
|    time_elapsed         | 5079        |
|    total_timesteps      | 499712      |
| train/                  |             |
|    approx_kl            | 0.010075958 |
|    clip_fraction        | 0.14        |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.5       |
|    explained_variance   | 0.0108      |
|    learning_rate        | 0.0003      |
|    loss                 | 48.4        |
|    n_updates            | 2430        |
|    policy_gradient_loss | -0.0156     |
|    std                  | 1.26        |
|    value_loss           | 154         |
-----------------------------------------


[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


-----------------------------------------
| time/                   |             |
|    fps                  | 98          |
|    iterations           | 245         |
|    time_elapsed         | 5099        |
|    total_timesteps      | 501760      |
| train/                  |             |
|    approx_kl            | 0.012109189 |
|    clip_fraction        | 0.159       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.5       |
|    explained_variance   | -0.0389     |
|    learning_rate        | 0.0003      |
|    loss                 | 73.9        |
|    n_updates            | 2440        |
|    policy_gradient_loss | -0.0155     |
|    std                  | 1.26        |
|    value_loss           | 162         |
-----------------------------------------


In [10]:
# Create env with domain randomization
env = RandomizedLiftEnv()

frames = []
obs, _ = env.reset()
done = False

while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    frame = env.env.sim.render(camera_name="frontview", width=640, height=480)
    frame = np.flip(frame, axis=0)   # flip because MuJoCo’s render is upside down
    frames.append(frame)

env.close()
imageio.mimsave("ppo_eval.mp4", frames, fps=30)
print("Evaluation finished, saved to ppo_eval.mp4")

[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /home/yifeng/work/env/mujoco/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


Evaluation finished, saved to ppo_eval.mp4
